In [1]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import shapely as shy
from shapely.ops import nearest_points
from shapely.geometry import Point, MultiPoint
import fiona
from scipy.spatial import cKDTree
import numpy as np

pd.options.display.max_rows = 4
%matplotlib inline
import os
os.environ["PROJ_LIB"] = "C:\ProgramData\Anaconda3\Library\share" #window

In [2]:
# ckdnearest function
# from "https://gist.github.com/jhconning/63a34a51acff83d116adc52308faf240"
def ckdnearest(gdA, gdB, bcol):
    """
    This function takes geodataframes: `gdA` and `gdB` and 
    a column name `bcol`. Both dataframes are assumed to have a `geometry` column. 
    It finds the nearest neighbor from each location in `gdA` to a 
    nearest neighbor in `gdB`.  

    It returns a two-column pandas dataframe with a 'distance' (here rounded to nearest foot)
    and the value of the `bcol` in `gdB'  (e.g. 'school_name')
    """
    
    nA = np.array(list(zip(gdA.geometry.x, gdA.geometry.y)) )
    nB = np.array(list(zip(gdB.geometry.x, gdB.geometry.y)) )
    btree = cKDTree(nB)
    dist, idx = btree.query(nA,k=1)
    df = pd.DataFrame.from_dict({'distance': dist.astype(int),
                             'bcol' : gdB.loc[idx, bcol].values })
    return df

In [3]:
#Filepaths
work = r"D:"
laptop = r"C:\Users\jerem"
origin = work

### Signal Intersection

In [4]:
# Open signal intersection stata dta file
fp = r"\Box Sync\Policy Evaluation\input_data\DOT_traffic_signals_Oct_2018\signal_controllers_clean.dta"
df = pd.read_stata(origin+fp)
df.head(3)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup
0,1.0,199793.609300002,986336.149,ALLEN STREET,CANAL STREET,,,0
1,2.0,202206.161899999,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0
2,3.0,201790.0942,982805.618399993,AVENUE OF THE AMERICAS,LISPENARD STREET,WEST BROADWAY,,0


In [5]:
# Convert coordinates into float
df['y'] = df['y'].astype(float)
df['x'] = df['x'].astype(float)
# Put the latitude and longtitude 
df['Coordinates'] = list(zip(df.x, df.y))
df['Coordinates'] = df['Coordinates'].apply(Point)
df1 = gpd.GeoDataFrame(df, geometry='Coordinates')

In [6]:
df1.crs = {'init' :'epsg:2263'}

### Leading Pedestrian Signal Interval

In [7]:
# Open LPIs shapefile
fp = r"\Box Sync\Policy Evaluation\input_data\VZV_Leading Pedestrian Interval Signals\geo_export_0c63b43f-83c0-4834-aa91-2c564c1bff2c.shp"
df2 = gpd.read_file(origin+fp)
df2.head(3)

,cross_stre,date_insta,time_insta,lat,long,main_stree,geometry
0,Dreiser loop East,2018-08-01,00:00:00.000,40.878465,-73.828273,Co-op City Blvd,POINT (-73.82827338235253 40.87846542795222)
1,West 119 St,2018-08-08,00:00:00.000,40.803958,-73.948271,Lenox Avenue,POINT (-73.94827133431397 40.80395846158952)
2,West 120 St,2018-08-08,00:00:00.000,40.804587,-73.947812,Lenox Avenue,POINT (-73.94781210658503 40.80458679730486)


In [8]:
df2.crs

{'init': 'epsg:4326'}

In [9]:
# Reproeject LPIS shapefile to EPSG 2263: NAD83 / New York Long Island (ftUS)
df2 = df2.to_crs({'init': 'epsg:2263'})

In [10]:
# Create ID variable
df2['LPIS_ID'] = df2.index
df2.head(3)

,cross_stre,date_insta,time_insta,lat,long,main_stree,geometry,LPIS_ID
0,Dreiser loop East,2018-08-01,00:00:00.000,40.878465,-73.828273,Co-op City Blvd,POINT (1031739.000154228 259373.000004255),0
1,West 119 St,2018-08-08,00:00:00.000,40.803958,-73.948271,Lenox Avenue,POINT (998570.9998893011 232184.9999184268),1
2,West 120 St,2018-08-08,00:00:00.000,40.804587,-73.947812,Lenox Avenue,POINT (998697.9999633889 232414.0001165908),2


In [11]:
df1[['distance_to_LPIS','nearest_LPIS']] = ckdnearest(df1, df2,'LPIS_ID')
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,nearest_LPIS
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,1915
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,2016


In [12]:
df1['intersection_id'].count()

13213

In [13]:
## Merge into df1 information from df2
df1 = df1.merge(df2[['date_insta','LPIS_ID']], how='left', left_on='nearest_LPIS', right_on='LPIS_ID', validate ="m:1")
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,nearest_LPIS,date_insta,LPIS_ID
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,1915,2015-10-16,1915
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,2016,1997-08-07,2016


In [14]:
df1['intersection_id'].count()

13213

In [15]:
df1.drop(['LPIS_ID'], axis=1, inplace=True)

## LION Street

In [16]:
# Open Streets shapefile
fp = r"\Box Sync\Policy Evaluation\input_data\nyclion_points_2263/nyclion_points_2263.shp"
df3 = gpd.read_file(origin+fp)
df3.head(3)

,OBJECTID,Street,SAFStreetN,FeatureTyp,SegmentTyp,IncExFlag,RB_Layer,NonPed,TrafDir,TrafSrc,...,LHi_Hyphen,RLo_Hyphen,RHi_Hyphen,FromLeft,ToLeft,FromRight,ToRight,Join_ID,SHAPE_Leng,geometry
0,1,EAST 168 STREET,None,0,U,None,B,None,T,DOT,...,699,596,716,599,699,596,716,2251001000000,396.030947,POINT (1010964.446978778 241812.2614034563)
1,1,EAST 168 STREET,None,0,U,None,B,None,T,DOT,...,699,596,716,599,699,596,716,2251001000000,396.030947,POINT (1011040.463264077 241747.2881445785)
2,1,EAST 168 STREET,None,0,U,None,B,None,T,DOT,...,699,596,716,599,699,596,716,2251001000000,396.030947,POINT (1011116.479549377 241682.3148857006)


In [17]:
# Create ID variable
df3['street_ID'] = df3.index

In [18]:
df3.crs

{'proj': 'lcc',
 'lat_1': 41.03333333333333,
 'lat_2': 40.66666666666666,
 'lat_0': 40.16666666666666,
 'lon_0': -74,
 'x_0': 300000.0000000001,
 'y_0': 0,
 'datum': 'NAD83',
 'units': 'us-ft',
 'no_defs': True}

In [19]:
df1[['distance_to_Street','nearest_Street']] = ckdnearest(df1, df3,'street_ID')
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,nearest_LPIS,date_insta,distance_to_Street,nearest_Street
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,1915,2015-10-16,7,375111
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,2016,1997-08-07,17,386723


In [20]:
df1['intersection_id'].count()

13213

In [21]:
## Merge into df1 information from df3
df1 = df1.merge(df3[['Number_Tra', 'Number_Par', 'Number_Tot', 'TrafDir','street_ID']], how='left', left_on='nearest_Street', right_on='street_ID', validate ="m:1")
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,nearest_LPIS,date_insta,distance_to_Street,nearest_Street,Number_Tra,Number_Par,Number_Tot,TrafDir,street_ID
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,1915,2015-10-16,7,375111,2,None,2,T,375111
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,2016,1997-08-07,17,386723,3,1,4,A,386723


In [22]:
df1['intersection_id'].count()

13213

In [23]:
df1.drop(['street_ID'], axis=1, inplace=True)

## Schools shapefile

In [24]:
# Open Schools shapefile
fp = r"\Box Sync\Policy Evaluation\input_data\school_private_public_2263\school_private_public_2263.shp"
df4 = gpd.read_file(origin+fp)
df4.head(3)

,uid,idagency,facname,opname,address,city,zipcode,bcode,facsubgrp,factype,capacity,util,xcoord,ycoord,geometry
0,192.0,321000145390,Our Lady Of Refuge School,Our Lady Of Refuge School,2708 Briggs Avenue,Bronx,10458,36005,Non-Public K-12 Schools,Elementary School - Non-public,NaN,267.0,1.014359e+06,254977.5176,POINT (1014358.6833 254977.5176)
1,294.0,332000226225,Yeshiva Toldos Yesuscher,Yeshiva Toldos Yesuscher,1531 63 Street,Brooklyn,11219,36047,Non-Public K-12 Schools,Elementary School - Non-public,NaN,74.0,9.857114e+05,166689.5140,POINT (985711.4182 166689.514)
2,2233.0,331400225670,Ohel Elozer,Ohel Elozer,263 Classon Ave-Ste 4b,Brooklyn,11205,36047,Non-Public K-12 Schools,High School - Non-public,NaN,161.0,9.951838e+05,191390.1394,POINT (995183.844 191390.1394)


In [25]:
df4.crs

{'proj': 'lcc',
 'lat_1': 41.03333333333333,
 'lat_2': 40.66666666666666,
 'lat_0': 40.16666666666666,
 'lon_0': -74,
 'x_0': 300000.0000000001,
 'y_0': 0,
 'datum': 'NAD83',
 'units': 'us-ft',
 'no_defs': True}

In [26]:
df1[['distance_to_Sch','nearest_Sch']] = ckdnearest(df1, df4,'uid')
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,nearest_LPIS,date_insta,distance_to_Street,nearest_Street,Number_Tra,Number_Par,Number_Tot,TrafDir,distance_to_Sch,nearest_Sch
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,1915,2015-10-16,7,375111,2,None,2,T,529,3433.0
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,2016,1997-08-07,17,386723,3,1,4,A,795,38150.0


## VZV Left turn calming

In [27]:
# Open VZV Left turn calming shapefile
fp = r"\Box Sync\Policy Evaluation\input_data\left_turn_traffic_calming_shapefile\left_turn_traffic_calming.shp"
df5 = gpd.read_file(origin+fp)
df5.head(3)

,treatment_,completion,geometry
0,Quick Kurb to X-walk,2017-12-08,POINT (-73.97604825017703 40.75144139179798)
1,"Daylighting, Box markings, (1) 6' Rubber speed...",2018-04-12,POINT (-73.82577379991478 40.75635805716421)
2,Quick Kurb to X-walk,2016-06-30,POINT (-73.97719688637932 40.7643236906894)


In [28]:
df5.crs

{'init': 'epsg:4326'}

In [29]:
# Reproeject VZV Left turn calming shapefile to EPSG 2263: NAD83 / New York Long Island (ftUS)
df5 = df5.to_crs({'init': 'epsg:2263'})
df5.crs

{'init': 'epsg:2263'}

In [30]:
# Create ID variable
df5['LTC_ID'] = df5.index
df5.head(3)

,treatment_,completion,geometry,LTC_ID
0,Quick Kurb to X-walk,2017-12-08,POINT (990886.2363875993 213047.939427546),0
1,"Daylighting, Box markings, (1) 6' Rubber speed...",2018-04-12,POINT (1032518.714146682 214886.3347008746),1
2,Quick Kurb to X-walk,2016-06-30,POINT (990566.7655698524 217741.2961825352),2


In [31]:
df1[['distance_to_LTC','nearest_LTC']] = ckdnearest(df1, df5,'LTC_ID')
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,...,distance_to_Street,nearest_Street,Number_Tra,Number_Par,Number_Tot,TrafDir,distance_to_Sch,nearest_Sch,distance_to_LTC,nearest_LTC
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,...,7,375111,2,None,2,T,529,3433.0,1183,191
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,...,17,386723,3,1,4,A,795,38150.0,411,77


In [32]:
## Merge into df1 information from df5
df1 = df1.merge(df5[['treatment_', 'completion', 'LTC_ID']], how='left', left_on='nearest_LTC', right_on='LTC_ID', validate ="m:1")
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,...,Number_Par,Number_Tot,TrafDir,distance_to_Sch,nearest_Sch,distance_to_LTC,nearest_LTC,treatment_,completion,LTC_ID
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,...,None,2,T,529,3433.0,1183,191,"Daylighting, Box markings, Pegatracks, Delinea...",2017-12-08,191
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,...,1,4,A,795,38150.0,411,77,Quick Kurb to x-walk,2017-12-08,77


In [33]:
df1['intersection_id'].count()

13213

In [34]:
df1.drop(['LTC_ID'], axis=1, inplace=True)

## VZV Street Improvement

In [35]:
# Open VZV Street Improvement shapefile
fp = r"\Box Sync\Policy Evaluation\input_data\street_improvement_proj_intersections_point\street_improvement_proj_intersections_point.shp"
df6 = gpd.read_file(origin+fp)
df6.head(3)

,Pjct_Name,SIP_YR,DateComplt,SIPProjTyp,geometry
0,Jackson Avenue at Pulaski Bridge,2009,2009-07-31,-,POINT (997734.4838460496 210053.6281589993)
1,12th Ave at W. 135th St,2009,2009-05-09,Traffic Network Chng - Pedestrian Safety,POINT (995894.2742718621 238501.8740837589)
2,Flatbush Ave at Church Ave,2009,2009-09-25,Pedestrian Safety - School Safety,POINT (995708.027925191 176232.7026909116)


In [36]:
df6.crs

{'proj': 'lcc',
 'lat_1': 41.03333333333333,
 'lat_2': 40.66666666666666,
 'lat_0': 40.16666666666666,
 'lon_0': -74,
 'x_0': 300000.0000000001,
 'y_0': 0,
 'datum': 'NAD83',
 'units': 'us-ft',
 'no_defs': True}

In [37]:
# Reproeject VZV Street Improvement shapefile to EPSG 2263: NAD83 / New York Long Island (ftUS)
df6 = df6.to_crs({'init': 'epsg:2263'})

In [38]:
df6.crs

{'init': 'epsg:2263'}

In [39]:
# Create ID variable
df6['StImpro_ID'] = df6.index
df6.head(3)

,Pjct_Name,SIP_YR,DateComplt,SIPProjTyp,geometry,StImpro_ID
0,Jackson Avenue at Pulaski Bridge,2009,2009-07-31,-,POINT (997734.4838460496 210053.6281589389),0
1,12th Ave at W. 135th St,2009,2009-05-09,Traffic Network Chng - Pedestrian Safety,POINT (995894.2742718621 238501.8740837032),1
2,Flatbush Ave at Church Ave,2009,2009-09-25,Pedestrian Safety - School Safety,POINT (995708.027925191 176232.7026908512),2


In [40]:
df1[['distance_to_StImpro','nearest_StImpro']] = ckdnearest(df1, df6,'StImpro_ID')
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,...,Number_Tot,TrafDir,distance_to_Sch,nearest_Sch,distance_to_LTC,nearest_LTC,treatment_,completion,distance_to_StImpro,nearest_StImpro
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,...,2,T,529,3433.0,1183,191,"Daylighting, Box markings, Pegatracks, Delinea...",2017-12-08,272,4
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,...,4,A,795,38150.0,411,77,Quick Kurb to x-walk,2017-12-08,2101,245


In [41]:
## Merge into df1 information from df5
df1 = df1.merge(df6[['DateComplt', 'SIPProjTyp', 'StImpro_ID']], how='left', left_on='nearest_StImpro', right_on='StImpro_ID', validate ="m:1")
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,...,nearest_Sch,distance_to_LTC,nearest_LTC,treatment_,completion,distance_to_StImpro,nearest_StImpro,DateComplt,SIPProjTyp,StImpro_ID
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,...,3433.0,1183,191,"Daylighting, Box markings, Pegatracks, Delinea...",2017-12-08,272,4,2009-10-31,-,4
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,...,38150.0,411,77,Quick Kurb to x-walk,2017-12-08,2101,245,2016-07-23,VZ Priority Geo - Traffic Calming,245


In [42]:
df1['intersection_id'].count()

13213

In [43]:
df1.drop(['StImpro_ID'], axis=1, inplace=True)

## Bike Route

In [44]:
# Open Bikeroute shapefile
fp = r"\Box Sync\Policy Evaluation\input_data\bike_routes_points_2263\bike_routes_points_2263.shp"
df7 = gpd.read_file(origin+fp)
df7.head(3)

,tf_facilit,comments,bikedir,ft_facilit,objectid_1,allclasses,date_instd,time_instd,lanecount,segment_id,boro,street,date_modda,time_modda,tostreet,fromstreet,onoffst,geometry
0,Protected Path,None,L,None,1.0,I,None,00:00:00.000,1.0,33547,1.0,9 AV,None,00:00:00.000,W 31 ST,W 16 ST,ON,POINT (984139.5802614246 211708.657997195)
1,Protected Path,None,L,None,1.0,I,None,00:00:00.000,1.0,33547,1.0,9 AV,None,00:00:00.000,W 31 ST,W 16 ST,ON,POINT (984140.0661334711 211709.5320271455)
2,Protected Path,None,L,None,1.0,I,None,00:00:00.000,1.0,33547,1.0,9 AV,None,00:00:00.000,W 31 ST,W 16 ST,ON,POINT (984140.5520055175 211710.4060570961)


In [45]:
df7.crs

{'proj': 'lcc',
 'lat_1': 41.03333333333333,
 'lat_2': 40.66666666666666,
 'lat_0': 40.16666666666666,
 'lon_0': -74,
 'x_0': 300000.0000000001,
 'y_0': 0,
 'datum': 'NAD83',
 'units': 'us-ft',
 'no_defs': True}

In [46]:
df1[['distance_to_bikeroute','nearest_bikeroute']] = ckdnearest(df1, df7,'objectid_1')
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,...,distance_to_LTC,nearest_LTC,treatment_,completion,distance_to_StImpro,nearest_StImpro,DateComplt,SIPProjTyp,distance_to_bikeroute,nearest_bikeroute
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,...,1183,191,"Daylighting, Box markings, Pegatracks, Delinea...",2017-12-08,272,4,2009-10-31,-,3,8553.0
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,...,411,77,Quick Kurb to x-walk,2017-12-08,2101,245,2016-07-23,VZ Priority Geo - Traffic Calming,230,1778.0


In [47]:
df1['nearest_bikeroute'].count()

13213

## Truck route

In [48]:
# Open Truckroute shapefile
fp = r"\Box Sync\Policy Evaluation\input_data\truckroute_points_2263\truckroute_points_2263.shp"
df8 = gpd.read_file(origin+fp)
df8.head(3)

,OBJECTID,Street,SAFStreetN,FeatureTyp,SegmentTyp,IncExFlag,RB_Layer,NonPed,TrafDir,TrafSrc,...,Restrictio,NYC_Reg,LtdLocal,ThruExwy,LocalBrg,LocalTunl,ThruBrg,ThruTunl,Shape_Leng,geometry
0,1,TRINITY PLACE,None,0,U,None,B,None,W,DOT,...,None,"NYCDOT Traffic Rules, Section 4-13-(d)(2)",None,None,None,None,None,None,61.384654,POINT (980855.2233999987 197527.1545999364)
1,2,WATER STREET,None,0,U,None,B,None,T,DOT,...,None,"NYCDOT Traffic Rules, Section 4-13-(d)(2)",None,None,None,None,None,None,243.748254,POINT (980827.536599993 195309.7279999424)
2,2,WATER STREET,None,0,U,None,B,None,T,DOT,...,None,"NYCDOT Traffic Rules, Section 4-13-(d)(2)",None,None,None,None,None,None,243.748254,POINT (980922.8409135042 195340.0114579148)


In [49]:
# Create ID variable
df8['truckroute_ID'] = df8.index
df8.head(3)

,OBJECTID,Street,SAFStreetN,FeatureTyp,SegmentTyp,IncExFlag,RB_Layer,NonPed,TrafDir,TrafSrc,...,NYC_Reg,LtdLocal,ThruExwy,LocalBrg,LocalTunl,ThruBrg,ThruTunl,Shape_Leng,geometry,truckroute_ID
0,1,TRINITY PLACE,None,0,U,None,B,None,W,DOT,...,"NYCDOT Traffic Rules, Section 4-13-(d)(2)",None,None,None,None,None,None,61.384654,POINT (980855.2233999987 197527.1545999364),0
1,2,WATER STREET,None,0,U,None,B,None,T,DOT,...,"NYCDOT Traffic Rules, Section 4-13-(d)(2)",None,None,None,None,None,None,243.748254,POINT (980827.536599993 195309.7279999424),1
2,2,WATER STREET,None,0,U,None,B,None,T,DOT,...,"NYCDOT Traffic Rules, Section 4-13-(d)(2)",None,None,None,None,None,None,243.748254,POINT (980922.8409135042 195340.0114579148),2


In [50]:
df8.crs

{'proj': 'lcc',
 'lat_1': 41.03333333333333,
 'lat_2': 40.66666666666666,
 'lat_0': 40.16666666666666,
 'lon_0': -74,
 'x_0': 300000.0000000001,
 'y_0': 0,
 'datum': 'NAD83',
 'units': 'us-ft',
 'no_defs': True}

In [51]:
df1[['distance_to_truckroute','nearest_truckroute']] = ckdnearest(df1, df8,'truckroute_ID')
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,...,treatment_,completion,distance_to_StImpro,nearest_StImpro,DateComplt,SIPProjTyp,distance_to_bikeroute,nearest_bikeroute,distance_to_truckroute,nearest_truckroute
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,...,"Daylighting, Box markings, Pegatracks, Delinea...",2017-12-08,272,4,2009-10-31,-,3,8553.0,26,2675
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,...,Quick Kurb to x-walk,2017-12-08,2101,245,2016-07-23,VZ Priority Geo - Traffic Calming,230,1778.0,17,5957


## Borough

In [52]:
# Open Borough shapefile
fp = r"\Box Sync\Policy Evaluation\input_data\nyc_boroughs_2263\nyc_boroughs_2263.shp"
df9 = gpd.read_file(origin+fp)
df9.head(3)

,bcode,bname,name,namelsad,geometry
0,36005,Bronx,Bronx,Bronx County,(POLYGON ((1008982.068976385 272752.8735210547...
1,36047,Brooklyn,Kings,Kings County,(POLYGON ((978869.3811487257 186863.7807399245...
2,36061,Manhattan,New York,New York County,(POLYGON ((1007701.483091666 258286.8905491272...


In [53]:
df9.crs = df1.crs
df1 = gpd.sjoin(df1,df9[['bname', 'geometry']], how='left', op='intersects')

In [54]:
# Remove index_right 
# https://gis.stackexchange.com/questions/265864/multiple-spatial-joins-with-geopandas-in-python?rq=1
df1.drop('index_right', axis=1, inplace=True)

In [55]:
df1['intersection_id'].count()

13213

In [56]:
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,...,completion,distance_to_StImpro,nearest_StImpro,DateComplt,SIPProjTyp,distance_to_bikeroute,nearest_bikeroute,distance_to_truckroute,nearest_truckroute,bname
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,...,2017-12-08,272,4,2009-10-31,-,3,8553.0,26,2675,Manhattan
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,...,2017-12-08,2101,245,2016-07-23,VZ Priority Geo - Traffic Calming,230,1778.0,17,5957,Manhattan


## Priority Intersections

In [57]:
# Priority intersection shapefile
fp = r"\Box Sync\Policy Evaluation\input_data\vz_priority_intersections_shapefile\vz_priority_intersections_shapefile\vz_priority_intersections.shp"
df10 = gpd.read_file(origin+fp)
df10.head(3)

,Street1,Street2,geometry
0,86 ST,5 AV,POINT (-74.02621402589912 40.62149167672204)
1,86 ST,4 AV,POINT (-74.02835031631516 40.6227986261606)
2,53 ST,3 AV,POINT (-74.01633774404843 40.64599588224035)


In [58]:
df10 = df10.to_crs({'init': 'epsg:2263'})

In [59]:
# Create ID variable
df10['priority_inters_ID'] = df10.index
df10.head(3)

,Street1,Street2,geometry,priority_inters_ID
0,86 ST,5 AV,POINT (976972.7937419016 165703.6459931984),0
1,86 ST,4 AV,POINT (976379.8979460699 166179.9869040358),1
2,53 ST,3 AV,POINT (979716.1872052844 174630.5208558975),2


In [60]:
df1[['distance_to_priorityinters','nearest_priorityinters']] = ckdnearest(df1, df10,'priority_inters_ID')
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,...,nearest_StImpro,DateComplt,SIPProjTyp,distance_to_bikeroute,nearest_bikeroute,distance_to_truckroute,nearest_truckroute,bname,distance_to_priorityinters,nearest_priorityinters
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,...,4,2009-10-31,-,3,8553.0,26,2675,Manhattan,7,245
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,...,245,2016-07-23,VZ Priority Geo - Traffic Calming,230,1778.0,17,5957,Manhattan,2289,191


## Safe Streets for seniors

In [61]:
# Safe Streets for seniors shapefile
fp = r"\Box Sync\Policy Evaluation\input_data\safe_streets_for_seniors_shapefile\safe_streets_for_seniors.shp"
df11 = gpd.read_file(origin+fp)
df11.head(3)

,Id,Name,Shape_Leng,Shape_Area,geometry
0,0,Astoria,14763.582784,8.306479e+06,POLYGON ((-73.91898633335025 40.76577129203545...
1,0,Bath Beach,15515.138903,1.187964e+07,"POLYGON ((-73.9915393446875 40.59039090158799,..."
2,0,Bay Ridge,21468.007265,1.475626e+07,POLYGON ((-74.03381765235082 40.61458678601677...


In [62]:
# Create ID variable
df11['seniors_ID'] = df11.index
df11.head(3)

,Id,Name,Shape_Leng,Shape_Area,geometry,seniors_ID
0,0,Astoria,14763.582784,8.306479e+06,POLYGON ((-73.91898633335025 40.76577129203545...,0
1,0,Bath Beach,15515.138903,1.187964e+07,"POLYGON ((-73.9915393446875 40.59039090158799,...",1
2,0,Bay Ridge,21468.007265,1.475626e+07,POLYGON ((-74.03381765235082 40.61458678601677...,2


In [63]:
df11 = df11.to_crs({'init': 'epsg:2263'})

In [64]:
df11['seniors_ID'].count()

41

In [65]:
df11.crs = df1.crs
df1 = gpd.sjoin(df1, df11[['Name', 'seniors_ID','geometry']], how='left', op='within')

In [66]:
# Remove index_right 
# https://gis.stackexchange.com/questions/265864/multiple-spatial-joins-with-geopandas-in-python?rq=1
df1.drop('index_right', axis=1, inplace=True)

In [67]:
df1['intersection_id'].count()

13213

In [68]:
df1.head(2)

,intersection_id,y,x,st1_name,st2_name,st3_name,st4_name,dup,Coordinates,distance_to_LPIS,...,SIPProjTyp,distance_to_bikeroute,nearest_bikeroute,distance_to_truckroute,nearest_truckroute,bname,distance_to_priorityinters,nearest_priorityinters,Name,seniors_ID
0,1.0,199793.6093,986336.149,ALLEN STREET,CANAL STREET,,,0,POINT (986336.149 199793.609300002),276,...,-,3,8553.0,26,2675,Manhattan,7,245,Lower East Side,27.0
1,2.0,202206.1619,982769.331,AVENUE OF THE AMERICAS,LAIGHT STREET,CANAL STREET,,0,POINT (982769.331 202206.161899999),592,...,VZ Priority Geo - Traffic Calming,230,1778.0,17,5957,Manhattan,2289,191,NaN,NaN


In [69]:
# Code writen by Prof. Conning
# https://github.com/jeremysze/LPIS/issues/5#issue-381767680
str_cols = list(df1.select_dtypes(include=['object']).columns)
for col in str_cols:
    df1[col] = df1[col].astype(str)

In [70]:
fp =  r"\Box Sync\Policy Evaluation\working_data\signal_intersection.dta"
df1.to_stata(origin+fp)